In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
 
spark = (SparkSession
         .builder
         .master("local")
         .appName("arrival")
         # Add MSSQL jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/mssql-jdbc-11.2.0.jre8.jar")
         .getOrCreate())
sc = spark.sparkContext

22/11/14 03:20:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
table = 'dbo.gps_log'
user = 'tsb'
password = 'Gps@tsb'
jdbcDF = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://119.59.114.113:1433;databaseName=tsb;encrypt=true;trustServerCertificate=true;") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

In [15]:
jdbcDF.printSchema()

root
 |-- device: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- datetime_gps: timestamp (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- speed: decimal(7,2) (nullable = true)
 |-- odometer: decimal(10,1) (nullable = true)
 |-- engine_status: integer (nullable = true)



In [3]:
jdbcDF.count()

150412250

In [23]:
3750/60

62.5

In [4]:
jdbcDF.orderBy(jdbcDF.datetime.desc()).limit(5).toPandas()

,device,datetime,datetime_gps,latitude,longitude,speed,odometer,engine_status
0,868998038201063,2022-10-17 16:56:19.893,2022-10-17 16:55:28,13.641630,100.410051,0.00,597793.0,1
1,864606045205555,2022-10-17 16:56:19.877,2022-10-17 16:55:17,13.787363,100.677485,0.00,7331207.0,0
2,868998037488265,2022-10-17 16:56:19.877,2022-10-17 16:55:29,13.762083,100.493501,5.00,1005730.0,1
3,868998037488828,2022-10-17 16:56:19.877,2022-10-17 16:55:28,13.745463,100.495605,9.00,1042340.0,1
4,868998038201063,2022-10-17 16:56:13.643,2022-10-17 16:55:28,13.641656,100.410086,0.00,597792.0,1


In [5]:
jdbcDF.orderBy(jdbcDF.datetime.asc()).limit(5).toPandas()

,device,datetime,datetime_gps,latitude,longitude,speed,odometer,engine_status
0,864352041902925,2022-09-17 19:17:50.087,2022-09-17 19:17:07,13.766008,100.539053,19.00,2207416.0,1
1,864352041904616,2022-09-17 19:17:50.337,2022-09-17 19:17:09,13.691606,100.487025,40.00,967888.0,1
2,864352041903188,2022-09-17 19:17:50.337,2022-09-17 19:16:41,13.662778,100.514278,0.00,995394.0,0
3,864352041903493,2022-09-17 19:17:50.337,2022-09-17 19:17:11,13.662981,100.514900,0.00,1232660.0,1
4,864352041905878,2022-09-17 19:17:50.353,2022-09-17 19:17:10,13.766218,100.535403,20.00,4860038.0,1


In [13]:
jdbcDF.orderBy(jdbcDF.datetime.asc()).toPandas().tocsv('/home/jovyan/work/data/gps_sit/gps_17_oct.csv', index=False)

22/10/17 10:41:57 ERROR Executor: Exception in task 97.0 in stage 7.0 (TID 104)]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3745)
	at java.base/java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:120)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:95)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:156)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:223)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.java:176)
	at java.base/java.io.DataOutputStream.write(DataOutputStream.java:107)
	at org.apache.spark.sql.catalyst.expressions.UnsafeRow.writeToStream(UnsafeRow.java:547)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:348)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda$2409/0x0000000841011040.apply(Unknown Source)
	at org.apache.spark.rdd.RDD.$anonfun$mapPar

Py4JJavaError: An error occurred while calling o1466.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 97 in stage 7.0 failed 1 times, most recent failure: Lost task 97.0 in stage 7.0 (TID 104) (c961deee2745 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3745)
	at java.base/java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:120)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:95)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:156)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:223)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.java:176)
	at java.base/java.io.DataOutputStream.write(DataOutputStream.java:107)
	at org.apache.spark.sql.catalyst.expressions.UnsafeRow.writeToStream(UnsafeRow.java:547)
	at org.apache.spark.sql.execution.SparkPlan.$anon

In [7]:
import pandas as pd
all_v = pd.read_csv('/home/jovyan/work/data/all_vehicle.csv')
all_v

,gps_imei,num,routes,chasis
0,868998031822683,16-5840 กทม.,35,MRSBCREM3MZM00001
1,864507039673982,16-5843 กทม.,35,MRSBCREM5MZM00033
2,868998030634774,16-5845 กทม.,35,MRSBCREM8MZM00057
3,868998030615351,16-5844 กทม.,35,MRSBCREM1MZM00062
4,868998030627521,16-5847 กทม.,35,MRSBCREM7MZM00034
...,...,...,...,...
452,867648045388376,16-3920,69,PRAT29A3UFR000476
453,867648045375092,16-3904,69,PRAT29A3UFR000477
454,867648045376983,16-3887,69,PRAT29A3UFR000478
455,867648045407127,16-3695,69,PRAT29A3UFR000439


In [9]:
gps_list = all_v.gps_imei.tolist()

In [12]:
import pyspark.sql.functions as f
jdbcDF.where(f.col('device').isin(gps_list)).show()

+------+--------+------------+--------+---------+-----+--------+-------------+
|device|datetime|datetime_gps|latitude|longitude|speed|odometer|engine_status|
+------+--------+------------+--------+---------+-----+--------+-------------+
+------+--------+------------+--------+---------+-----+--------+-------------+



In [25]:
import pandas as pd
df=pd.read_csv('/home/jovyan/work/data/spark_files/data_gps_1000000.csv')
df

,nrow,device,datetime,datetime_gps,latitude,longitude,speed,odometer,engine_status
0,1,864352041902925,2022-09-17 19:17:50.087,2022-09-17 19:17:07,13.766008,100.539053,19.0,2207416.0,1
1,2,864352041904616,2022-09-17 19:17:50.337,2022-09-17 19:17:09,13.691606,100.487025,40.0,967888.0,1
2,3,864352041903188,2022-09-17 19:17:50.337,2022-09-17 19:16:41,13.662778,100.514278,0.0,995394.0,0
3,4,864352041903493,2022-09-17 19:17:50.337,2022-09-17 19:17:11,13.662981,100.514900,0.0,1232660.0,1
4,5,864352041912882,2022-09-17 19:17:50.353,2022-09-17 19:16:43,13.786936,100.678288,0.0,3299446.0,0
...,...,...,...,...,...,...,...,...,...
999995,999996,864352041921479,2022-09-18 21:23:29.987,2022-09-18 21:22:43,13.662775,100.514463,0.0,562642.0,0
999996,999997,864352043372945,2022-09-18 21:23:29.987,2022-09-18 21:22:39,13.662795,100.514265,0.0,1562013.0,0
999997,999998,864352043374172,2022-09-18 21:23:30.003,2022-09-18 21:22:56,13.781488,100.545118,30.0,3669950.0,1
999998,999999,864352043373521,2022-09-18 21:23:30.003,2022-09-18 21:22:56,13.787013,100.678375,0.0,3188712.0,1


In [5]:
from datetime import datetime
a = pd.date_range('2022-11-14', periods=17).date
a = [i.strftime('%Y-%m-%d') for i in a]
a

['2022-11-14',
 '2022-11-15',
 '2022-11-16',
 '2022-11-17',
 '2022-11-18',
 '2022-11-19',
 '2022-11-20',
 '2022-11-21',
 '2022-11-22',
 '2022-11-23',
 '2022-11-24',
 '2022-11-25',
 '2022-11-26',
 '2022-11-27',
 '2022-11-28',
 '2022-11-29',
 '2022-11-30']